## Spatial Pyramid

In [2]:
from utils import load_my_data
from utils import extract_DenseSift_descriptors
from utils import build_codebook
from utils import input_vector_encoder
from classifier import svm_classifier
from keras.datasets import mnist
import spm

import numpy as np
import argparse

/Users/karanuppal/anaconda/envs/gl-env/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/karanuppal/anaconda/envs/gl-env/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.
/Users/karanuppal/anaconda/envs/gl-env/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be 

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
print X_train.shape

(60000, 28, 28)


In [5]:


X, y = X_train, y_train

X_small = []
y_small = []
for digit in range(0,10):
    x_tmp = [X[i] for i in range(60000) if y[i] == digit]
    y_tmp = [y[i] for i in range(60000) if y[i] == digit]
    X_small = X_small + x_tmp[:100]
    y_small = y_small + y_tmp[:100]

X = X_small
y = y_small

print "Codebook Size: {:d}".format(spm.VOC_SIZE)
print "Pyramid level: {:d}".format(spm.PYRAMID_LEVEL)
print "Dense SIFT feature extraction"

Codebook Size: 200
Pyramid level: 2
Dense SIFT feature extraction


In [6]:
x_feature = [extract_DenseSift_descriptors(img) for img in X]
x_kp, x_des = zip(*x_feature)
xt_feature = [extract_DenseSift_descriptors(img) for img in X_test]
xt_kp, xt_des = zip(*xt_feature)

In [7]:
%%time
## SIFT based matching only for experimentation
trx = np.array([np.array(a).flatten() for a in x_des])
tex = np.array([np.array(a).flatten() for a in xt_des])
svm_classifier(trx, y, tex, y_test)

classifier.py:19: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if x_test == None and y_test == None:


Tuning hyper-parameters

Best parameters set found on development set:

SVC(C=0.001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Grid scores on development set:

0.300 (+/-0.037) for {'C': 0.001, 'gamma': 0.001}
0.102 (+/-0.005) for {'C': 0.001, 'gamma': 0.01}
0.100 (+/-0.000) for {'C': 0.001, 'gamma': 0.1}
0.100 (+/-0.000) for {'C': 0.001, 'gamma': 1.0}
0.100 (+/-0.000) for {'C': 0.001, 'gamma': 10.0}
0.100 (+/-0.000) for {'C': 0.001, 'gamma': 100.0}
0.300 (+/-0.037) for {'C': 0.01, 'gamma': 0.001}
0.102 (+/-0.005) for {'C': 0.01, 'gamma': 0.01}
0.100 (+/-0.000) for {'C': 0.01, 'gamma': 0.1}
0.100 (+/-0.000) for {'C': 0.01, 'gamma': 1.0}
0.100 (+/-0.000) for {'C': 0.01, 'gamma': 10.0}
0.100 (+/-0.000) for {'C': 0.01, 'gamma': 100.0}
0.300 (+/-0.037) for {'C': 0.1, 'gamma': 0.001}
0.102 (+/-0.005) for {'C': 0.1, 'gamma'

In [8]:
%%time
print "Building the codebook, it will take some time"
codebook = build_codebook(x_des, spm.VOC_SIZE)
#import cPickle
#with open('./data/codebook_spm.pkl','w') as f:
#    cPickle.dump(codebook, f)

Building the codebook, it will take some time
CPU times: user 768 ms, sys: 869 ms, total: 1.64 s
Wall time: 1min 35s


In [9]:
%%time

print "Spatial Pyramid Matching encoding"
X = [spm.spatial_pyramid_matching(X[i],
                              x_des[i],
                              codebook,
                              level=spm.PYRAMID_LEVEL)
                              for i in xrange(len(x_des))]

Spatial Pyramid Matching encoding
CPU times: user 5.49 s, sys: 262 ms, total: 5.75 s
Wall time: 3.6 s


In [10]:
%%time
X_test = [spm.spatial_pyramid_matching(X_test[i],
                              xt_des[i],
                              codebook,
                              level=spm.PYRAMID_LEVEL)
                              for i in xrange(len(xt_des))]

CPU times: user 51.6 s, sys: 1.58 s, total: 53.1 s
Wall time: 29.1 s


In [11]:
%%time
X = np.asarray(X)
X_test = np.asarray(X_test)
svm_classifier(X, y, X_test, y_test)

Tuning hyper-parameters

Best parameters set found on development set:

SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1.0, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Grid scores on development set:

0.793 (+/-0.107) for {'C': 0.001, 'gamma': 0.001}
0.793 (+/-0.107) for {'C': 0.001, 'gamma': 0.01}
0.796 (+/-0.105) for {'C': 0.001, 'gamma': 0.1}
0.813 (+/-0.085) for {'C': 0.001, 'gamma': 1.0}
0.613 (+/-0.077) for {'C': 0.001, 'gamma': 10.0}
0.135 (+/-0.028) for {'C': 0.001, 'gamma': 100.0}
0.793 (+/-0.107) for {'C': 0.01, 'gamma': 0.001}
0.793 (+/-0.107) for {'C': 0.01, 'gamma': 0.01}
0.796 (+/-0.105) for {'C': 0.01, 'gamma': 0.1}
0.813 (+/-0.085) for {'C': 0.01, 'gamma': 1.0}
0.613 (+/-0.077) for {'C': 0.01, 'gamma': 10.0}
0.135 (+/-0.028) for {'C': 0.01, 'gamma': 100.0}
0.793 (+/-0.107) for {'C': 0.1, 'gamma': 0.001}
0.793 (+/-0.107) for {'C': 0.1, 'gamma': 0